## Embeddings

En nuestro ejemplo anterior, trabajamos con vectores de bolsa de palabras de alta dimensión con una longitud de `vocab_size`, y estábamos convirtiendo explícitamente desde vectores de representación posicional de baja dimensión a una representación dispersa de una sola posición activa (one-hot). Esta representación de una sola posición activa no es eficiente en términos de memoria, además, cada palabra se trata de manera independiente, es decir, los vectores codificados en una sola posición activa no expresan ninguna similitud semántica entre las palabras.

En esta unidad, continuaremos explorando el dataset **News AG**. Para comenzar, carguemos los datos y obtengamos algunas definiciones del cuaderno anterior.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## ¿Qué es el embedding?

La idea del **embedding** es representar palabras mediante vectores densos de menor dimensión, que de alguna manera reflejen el significado semántico de una palabra. Más adelante discutiremos cómo construir embeddings de palabras significativos, pero por ahora pensemos en los embeddings simplemente como una forma de reducir la dimensionalidad de un vector de palabras.

Entonces, una capa de embedding tomaría una palabra como entrada y produciría un vector de salida con un `embedding_size` especificado. En cierto sentido, es muy similar a una capa `Linear`, pero en lugar de tomar un vector codificado en one-hot, podrá tomar un número de palabra como entrada.

Al usar una capa de embedding como la primera capa en nuestra red, podemos cambiar del modelo de bolsa de palabras (**bag-of-words**) al modelo de **bolsa de embeddings** (**embedding bag**), donde primero convertimos cada palabra de nuestro texto en su embedding correspondiente, y luego calculamos alguna función de agregación sobre todos esos embeddings, como `sum`, `average` o `max`.

![Imagen que muestra un clasificador con embedding para cinco palabras en secuencia.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

Nuestra red neuronal clasificador comenzará con una capa de embedding, luego una capa de agregación, y finalmente un clasificador lineal en la parte superior:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Cómo manejar el tamaño variable de las secuencias

Como resultado de esta arquitectura, los minibatches para nuestra red necesitarán ser creados de una manera específica. En la unidad anterior, al usar bag-of-words, todos los tensores BoW en un minibatch tenían el mismo tamaño `vocab_size`, independientemente de la longitud real de nuestra secuencia de texto. Una vez que pasamos a las incrustaciones de palabras, terminaremos con un número variable de palabras en cada muestra de texto, y al combinar esas muestras en minibatches tendremos que aplicar algún tipo de relleno.

Esto se puede hacer utilizando la misma técnica de proporcionar la función `collate_fn` a la fuente de datos:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Entrenando el clasificador de incrustaciones

Ahora que hemos definido un dataloader adecuado, podemos entrenar el modelo utilizando la función de entrenamiento que definimos en la unidad anterior:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Nota**: Aquí solo estamos entrenando con 25k registros (menos de una época completa) por cuestiones de tiempo, pero puedes continuar entrenando, escribir una función para entrenar durante varias épocas y experimentar con el parámetro de la tasa de aprendizaje para lograr una mayor precisión. Deberías poder alcanzar una precisión de alrededor del 90%.


### Capa EmbeddingBag y Representación de Secuencias de Longitud Variable

En la arquitectura anterior, necesitábamos rellenar todas las secuencias para que tuvieran la misma longitud y así poder ajustarlas en un minibatch. Esta no es la forma más eficiente de representar secuencias de longitud variable; otra opción sería usar un vector de **desplazamientos** (offset), que contendría los desplazamientos de todas las secuencias almacenadas en un único vector grande.

![Imagen que muestra una representación de secuencia con desplazamientos](../../../../../lessons/5-NLP/14-Embeddings/images/offset-sequence-representation.png)

> **Nota**: En la imagen de arriba, mostramos una secuencia de caracteres, pero en nuestro ejemplo estamos trabajando con secuencias de palabras. Sin embargo, el principio general de representar secuencias con un vector de desplazamientos sigue siendo el mismo.

Para trabajar con la representación basada en desplazamientos, usamos la capa [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Es similar a `Embedding`, pero toma como entrada un vector de contenido y un vector de desplazamientos, e incluye además una capa de promediado, que puede ser `mean`, `sum` o `max`.

Aquí está la red modificada que utiliza `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Para preparar el conjunto de datos para el entrenamiento, necesitamos proporcionar una función de conversión que prepare el vector de desplazamiento:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Nota que, a diferencia de todos los ejemplos anteriores, nuestra red ahora acepta dos parámetros: vector de datos y vector de desplazamiento, que son de diferentes tamaños. De manera similar, nuestro cargador de datos también nos proporciona 3 valores en lugar de 2: tanto los vectores de texto como los vectores de desplazamiento se proporcionan como características. Por lo tanto, necesitamos ajustar ligeramente nuestra función de entrenamiento para encargarnos de eso:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Embeddings Semánticos: Word2Vec

En nuestro ejemplo anterior, la capa de embeddings del modelo aprendió a mapear palabras a representaciones vectoriales, sin embargo, esta representación no tenía mucho significado semántico. Sería ideal aprender una representación vectorial en la que palabras similares o sinónimos correspondan a vectores cercanos entre sí en términos de alguna distancia vectorial (por ejemplo, distancia euclidiana).

Para lograr esto, necesitamos preentrenar nuestro modelo de embeddings en una gran colección de texto de una manera específica. Una de las primeras formas de entrenar embeddings semánticos se llama [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Se basa en dos arquitecturas principales que se utilizan para producir una representación distribuida de palabras:

 - **Continuous bag-of-words** (CBoW) — en esta arquitectura, entrenamos el modelo para predecir una palabra a partir del contexto circundante. Dado el ngrama $(W_{-2},W_{-1},W_0,W_1,W_2)$, el objetivo del modelo es predecir $W_0$ a partir de $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** es lo opuesto a CBoW. El modelo utiliza una ventana de palabras de contexto circundantes para predecir la palabra actual.

CBoW es más rápido, mientras que skip-gram es más lento, pero hace un mejor trabajo representando palabras poco frecuentes.

![Imagen que muestra los algoritmos CBoW y Skip-Gram para convertir palabras en vectores.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Para experimentar con embeddings Word2Vec preentrenados en el conjunto de datos de Google News, podemos usar la biblioteca **gensim**. A continuación, encontramos las palabras más similares a 'neural'.

> **Nota:** ¡Cuando creas vectores de palabras por primera vez, descargarlos puede tomar algo de tiempo!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Podemos también calcular incrustaciones vectoriales a partir de la palabra, para ser utilizadas en el entrenamiento del modelo de clasificación (solo mostramos los primeros 20 componentes del vector para mayor claridad):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Lo grandioso de las incrustaciones semánticas es que puedes manipular la codificación vectorial para cambiar la semántica. Por ejemplo, podemos pedir encontrar una palabra cuya representación vectorial sea lo más cercana posible a las palabras *rey* y *mujer*, y lo más alejada posible de la palabra *hombre*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Tanto CBoW como Skip-Grams son incrustaciones "predictivas", ya que solo toman en cuenta los contextos locales. Word2Vec no aprovecha el contexto global.

**FastText** se basa en Word2Vec al aprender representaciones vectoriales para cada palabra y los n-gramas de caracteres que se encuentran dentro de cada palabra. Los valores de las representaciones se promedian en un vector en cada paso de entrenamiento. Aunque esto agrega mucha computación adicional al pre-entrenamiento, permite que las incrustaciones de palabras codifiquen información de sub-palabras.

Otro método, **GloVe**, aprovecha la idea de la matriz de co-ocurrencia y utiliza métodos neuronales para descomponer la matriz de co-ocurrencia en vectores de palabras más expresivos y no lineales.

Puedes experimentar con el ejemplo cambiando las incrustaciones a FastText y GloVe, ya que gensim admite varios modelos diferentes de incrustaciones de palabras.


## Uso de Embeddings Preentrenados en PyTorch

Podemos modificar el ejemplo anterior para prellenar la matriz en nuestra capa de embedding con embeddings semánticos, como Word2Vec. Debemos tener en cuenta que los vocabularios del embedding preentrenado y nuestro corpus de texto probablemente no coincidan, por lo que inicializaremos los pesos de las palabras faltantes con valores aleatorios:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Ahora entrenemos nuestro modelo. Ten en cuenta que el tiempo que lleva entrenar el modelo es significativamente mayor que en el ejemplo anterior, debido al tamaño más grande de la capa de incrustación y, por lo tanto, a un número mucho mayor de parámetros. Además, debido a esto, es posible que necesitemos entrenar nuestro modelo con más ejemplos si queremos evitar el sobreajuste.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

En nuestro caso, no observamos un gran aumento en la precisión, lo cual probablemente se deba a vocabularios bastante diferentes.  
Para superar el problema de los vocabularios distintos, podemos usar una de las siguientes soluciones:  
* Reentrenar el modelo word2vec con nuestro vocabulario  
* Cargar nuestro conjunto de datos utilizando el vocabulario del modelo word2vec preentrenado. El vocabulario usado para cargar el conjunto de datos puede especificarse durante la carga.  

El último enfoque parece más sencillo, especialmente porque el marco `torchtext` de PyTorch contiene soporte integrado para embeddings. Por ejemplo, podemos instanciar un vocabulario basado en GloVe de la siguiente manera:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


El vocabulario cargado tiene las siguientes operaciones básicas:  
* El diccionario `vocab.stoi` nos permite convertir una palabra en su índice dentro del diccionario.  
* `vocab.itos` hace lo contrario: convierte un número en una palabra.  
* `vocab.vectors` es el array de vectores de incrustación, por lo que, para obtener la incrustación de una palabra `s`, necesitamos usar `vocab.vectors[vocab.stoi[s]]`.  

Aquí tienes un ejemplo de cómo manipular las incrustaciones para demostrar la ecuación **amable-hombre+mujer = reina** (tuve que ajustar un poco el coeficiente para que funcionara):  


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Para entrenar el clasificador utilizando esos embeddings, primero necesitamos codificar nuestro conjunto de datos utilizando el vocabulario de GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Como hemos visto anteriormente, todas las incrustaciones de vectores se almacenan en la matriz `vocab.vectors`. Esto hace que sea muy fácil cargar esos pesos en los pesos de la capa de incrustación mediante una simple copia:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Una de las razones por las que no estamos viendo un aumento significativo en la precisión se debe al hecho de que algunas palabras de nuestro conjunto de datos faltan en el vocabulario preentrenado de GloVe, y por lo tanto, esencialmente se ignoran. Para superar este hecho, podemos entrenar nuestras propias incrustaciones en nuestro conjunto de datos.


## Embeddings Contextuales

Una limitación clave de las representaciones tradicionales de embeddings preentrenados, como Word2Vec, es el problema de la desambiguación del sentido de las palabras. Aunque los embeddings preentrenados pueden capturar parte del significado de las palabras en contexto, todos los posibles significados de una palabra se codifican en el mismo embedding. Esto puede generar problemas en los modelos posteriores, ya que muchas palabras, como la palabra 'play', tienen diferentes significados dependiendo del contexto en el que se utilicen.

Por ejemplo, la palabra 'play' en estas dos frases tiene significados bastante diferentes:
- Fui a una **obra** en el teatro.
- John quiere **jugar** con sus amigos.

Los embeddings preentrenados mencionados anteriormente representan ambos significados de la palabra 'play' en el mismo embedding. Para superar esta limitación, necesitamos construir embeddings basados en el **modelo de lenguaje**, que se entrena con un gran corpus de texto y *sabe* cómo las palabras pueden combinarse en diferentes contextos. Hablar de embeddings contextuales está fuera del alcance de este tutorial, pero volveremos a ellos cuando hablemos de modelos de lenguaje en la próxima unidad.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
